**1. Discuss what type of sequence prediction approach (sequence-to-vector, sequence-to-sequence, or encoder-decoder) is most sensible to predict the operating mode of a turbine based on the two sensor reading time series. Also describe what data shape you need to use for your chosen approach.**

The most sensible approach for predicting the operating mode of a turbine based on the two sensor reading time series would be a sequence-to-vector approach. This is because we have two time series inputs (time_series_1 and time_series_2), and we want to predict a single output (operating mode) for each time series pair.


For this approach, we could use a combination of 1D convolutional layers (Conv1D) and recurrent layers (such as LSTM or GRU) to capture the patterns in the time series data. The final output of the model would be a dense layer with softmax activation to predict the operating mode (0, 1, 2, or 3).




**4. We have come across Conv1D layers as a tool for analyzing time series. Different from recurrent layers such as SimpleRNN, LSTM, or GRU, when we apply a Conv1D layer to a part of a sequence, the operation does not depend on the application of the layer to previous parts of the sequence. Discuss in which types of (business) applications ConviD layers can be particularly useful, and in which you should prefer a recurrent layer.**


**Conv1D layers** can be particularly useful in (business) applications where there are:


**Signal processing**: Conv1D layers can help in analyzing and processing audio signals, sensor data, and other one-dimensional signals.
**Text analysis**: When it comes to images, the filters of a 2D convolution layer move across both the height and width dimensions. However, Conv1D only traverses a single axis, making it appropriate for applying convolution to sequential data, such as text or signals. Consequently, using Conv1D for these types of data makes perfect sense.


**We should prefer a recurrent layer in situations where**:
**Prediction problems**

When it comes to solving prediction problems involving sequences, Recurrent Layers are typically very useful. Sequence prediction problems can take many forms and are typically characterized by the types of inputs and outputs that they involve.

**Machine Translation**

In the case of machine translation, some form of Recurrent Layer can be leveraged to translate text from one language to another. Almost all modern translation systems incorporate some advanced variant of a Recurrent Neural Network (RNN). In this setup, the source language can serve as the input, while the desired output will be the corresponding text in the target language.


In [3]:
import pandas as np
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [4]:
with open('/time_series_1.pickle', 'rb') as handle:
  time_series_1 = pickle.load(handle)

with open('/time_series_2.pickle', 'rb') as handle:
  time_series_2 = pickle.load(handle)

with open('/y.pickle', 'rb') as handle:
  y = pickle. load(handle)

In [5]:
time_series_1

array([[30, 29, 29, ..., 36, 35, 50],
       [31, 31, 30, ..., 37, 32, 31],
       [28, 28, 28, ..., 40, 29, 38],
       ...,
       [28, 28, 28, ..., 38, 28, 36],
       [28, 28, 29, ..., 31, 36, 51],
       [31, 31, 30, ..., 35, 38, 24]], dtype=uint8)

In [6]:
time_series_2

array([[84, 72, 93, ...,  2,  2,  1],
       [71, 83, 81, ...,  1,  1,  1],
       [78, 71, 82, ...,  0,  0,  0],
       ...,
       [83, 72, 95, ...,  0,  0,  0],
       [84, 68, 92, ...,  1,  0,  0],
       [81, 71, 98, ...,  0,  0,  0]], dtype=uint8)

In [7]:
y

array([1., 2., 3., ..., 3., 3., 1.])

In [8]:

# Combine time series data
X = np.stack((time_series_1, time_series_2), axis=-1)

# Convert y to one-hot encoding
y = tf.keras.utils.to_categorical(y, num_classes=4)


In [9]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify = y)

In [10]:
# Define batch size and shuffle buffer size
batch_size = 32
shuffle_buffer_size = len(y_train)

In [11]:
# Create a TensorFlow dataset from the train set
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.shuffle(shuffle_buffer_size).batch(batch_size)

# Create a TensorFlow dataset from the test set
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))
test_dataset = test_dataset.batch(batch_size)

## **Baseline Model**

In [12]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(filters=32, kernel_size=5, activation='relu', input_shape=(5000, 2)),
    tf.keras.layers.SimpleRNN(units=64, dropout=0.2, recurrent_dropout=0.2, return_sequences=True),
    tf.keras.layers.GRU(units=64, dropout=0.2, recurrent_dropout=0.2),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')
])

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# For creating the checkpoint for out model so that we can save the optimal model.
filepath="/content/drive/MyDrive/workspace/sarthak/baseline_model.hdf5"

checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

callbacks_list = [checkpoint]

In [13]:
history = model.fit(train_dataset, epochs=10, validation_data=test_dataset, verbose=1, callbacks=callbacks_list)

Epoch 1/10
100/100 [==============================] - ETA: 0s - loss: 1.3843 - accuracy: 0.2853 
Epoch 1: val_loss improved from inf to 1.33122, saving model to /content/drive/MyDrive/workspace/sarthak/baseline_model.hdf5
100/100 [==============================] - 1614s 16s/step - loss: 1.3843 - accuracy: 0.2853 - val_loss: 1.3312 - val_accuracy: 0.3663
Epoch 2/10
100/100 [==============================] - ETA: 0s - loss: 1.3459 - accuracy: 0.3319 
Epoch 2: val_loss improved from 1.33122 to 1.26288, saving model to /content/drive/MyDrive/workspace/sarthak/baseline_model.hdf5
100/100 [==============================] - 1622s 16s/step - loss: 1.3459 - accuracy: 0.3319 - val_loss: 1.2629 - val_accuracy: 0.4425
Epoch 3/10
100/100 [==============================] - ETA: 0s - loss: 1.3080 - accuracy: 0.3731 
Epoch 3: val_loss improved from 1.26288 to 1.20639, saving model to /content/drive/MyDrive/workspace/sarthak/baseline_model.hdf5
100/100 [==============================] - 1620s 16s/step 

In [14]:
test_loss, test_acc = model.evaluate(test_dataset)
print('Test accuracy', test_acc)
print('Test loss', test_loss)

25/25 [==============================] - 35s 1s/step - loss: 0.8038 - accuracy: 0.6612
Test accuracy 0.6612499952316284
Test loss 0.803802490234375
